In [ ]:
!pip install kafka-python
from kafka import KafkaConsumer
import pandas as pd
import numpy as np
from sklearn import metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from matplotlib import pyplot


def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)


def rank_feature_importance(m, type="Logistic Regression"):
  if type == "Random Forest":
    importance = m.feature_importances_
  else:
    importance = m.coef_[0]

  for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
  # plot feature importance
  pyplot.bar([x for x in range(len(importance))], importance)
  pyplot.show()

df = pd.read_csv('dataset_sdn.csv')

unneeded_features = ['byteperflow','tot_dur','packetins', 'tx_bytes', 'rx_bytes','pktperflow', 'pktrate', 'src', 'dst', 'Protocol', 'dt', 'switch', 'dur', 'dur_nsec', 'flows', 'Pairflow', 'port_no', 'tx_kbps', 'rx_kbps', 'tot_kbps']

for i in unneeded_features: del df[i]

print(df)


df.dropna(inplace=True)
indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)

df = df[indices_to_keep].astype(np.float64)
x = df.drop('label',axis = 1)
y = df.label
x_train, x_test, y_train, y_test = train_test_split(x, y)

#from sklearn.preprocessing import MinMaxScaler
#scaling = MinMaxScaler(feature_range=(-1,1)).fit(x_train)
#x_train = scaling.transform(x_train)
#x_test = scaling.transform(x_test)


from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier()

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, y_train)


        pktcount  bytecount  label
0          45304   48294064      0
1         126395  134737070      0
2          90333   96294978      0
3          90333   96294978      0
4          90333   96294978      0
...          ...        ...    ...
104340        79       7742      0
104341        79       7742      0
104342        31       3038      0
104343        31       3038      0
104344        31       3038      0

[104345 rows x 3 columns]


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_pred)

accuracy_percentage = 100 * accuracy
print("Random Forest Accuracy:", accuracy_percentage)

Random Forest Accuracy: 99.85050024916625


In [ ]:
print(x_train)

       pktcount   bytecount
16698   32933.0  35106578.0
19895   43798.0  46688668.0
2376    95620.0  99636040.0
68936     712.0     69776.0
29318    7581.0   8081346.0
...         ...         ...
90626     795.0     77910.0
46234      30.0      2940.0
45271  185171.0   9999234.0
82487   30446.0  33613324.0
70839   94758.0   5116932.0

[78258 rows x 2 columns]


In [ ]:
print(x_test[0].type)

AttributeError: ignored

In [ ]:
!pip install kafka-python
from kafka import KafkaConsumer

In [ ]:
import json
consumer = KafkaConsumer (bootstrap_servers="129.114.25.94:30000")
consumer.subscribe (topics=["utilization1", "durations", "counts_flows", "packet_ins", "tx_rx"])

# we keep reading and printing
for msg in consumer:
  j = json.loads(msg.value)
  prediction_job = np.array([j['1'][0]['byte_count'], j['1'][0]['packet_count']])
  prediction_job = prediction_job.reshape(1, -1)
  print(prediction_job)
  y_pred = clf.predict(prediction_job)
  print(y_pred)


KeyboardInterrupt: ignored